In [219]:
# import python modules 
import requests
import time
import urllib3
import pprint
#urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import json

nb_url = "http://192.168.28.79"
headers = {'Content-Type': 'application/json', 'Accept': 'application/json'} 

In [220]:
username = "gongdaiAdmin"
password = "1101810209"

body = {
    "username" : username,      
    "password" : password  
}

login_URL = nb_url + "/ServicesAPI/API/V1/Session"

def login(login_URL, body, headers):
    try:
        # Do the HTTP request
        response = requests.post(login_URL, headers=headers, data = json.dumps(body), verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js["token"])
        else:
            return ("Get token failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
token = login(login_URL, body, headers)
print(token)

f986d4f5-eeb5-4e3b-ac99-6e246c065838


In [221]:
Accessible_tenants_url = nb_url + "/ServicesAPI/API/V1/CMDB/Tenants"

def get_all_accessible_tenants(Accessible_tenants_url, token, headers):
    headers["Token"] = token
    try:
        # Do the HTTP request
        response = requests.get(Accessible_tenants_url, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            tenants = result["tenants"]
            tenant =  [x for x in tenants if x["tenantName"] == "Initial Tenant"]
            #tj = tenant.json()
            tenantId = tenant[0]["tenantId"]
            #ten_j = json.dumps(tenant)
            return tenantId
        else:
            return ("Get tenants failed! - " + str(response.text))
    except Exception as e: return (e)

tenantId = get_all_accessible_tenants(Accessible_tenants_url, token, headers)
print(tenantId)

fb24f3f0-81a7-1929-4b8f-99106c23fa5b


In [222]:
Accessible_domains_url = nb_url + "/ServicesAPI/API/V1/CMDB/Domains"

def get_all_accessible_domains(tenantId, token, headers):
    headers["Token"] = token
    data = {"tenantId": tenantId}
    try:
        # Do the HTTP request
        response = requests.get(Accessible_domains_url, params=data, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            domains = result["domains"]
            domain =  [x for x in domains if x["domainName"] == "GE Test"]
            domainId = domain[0]["domainId"]
            return domainId
        else:
            return ("Get domains failed! - " + str(response.text))

    except Exception as e: print (str(e))

domainId = get_all_accessible_domains(tenantId, token, headers)
print(domainId)

0201adc4-ae96-46f0-ae3d-01cdba9e41d6


In [223]:
Specify_a_working_domain_url = nb_url + "/ServicesAPI/API/V1/Session/CurrentDomain"

def specify_a_working_domain(tenantId, domainId, Specify_a_working_domain_url, headers, token):
    headers["Token"] = token
    body = {
        "tenantId": tenantId,
        "domainId": domainId
    }
    
    try:
        # Do the HTTP request
        response = requests.put(Specify_a_working_domain_url, data=json.dumps(body), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return ("Working Domain Specified Successfully, with domainId:" + domainId)
            
        elif response.status_code != 200:
            return ("Login failed! - " + str(response.text))

    except Exception as e: print (str(e))

res =  specify_a_working_domain(tenantId, domainId, Specify_a_working_domain_url, headers, token)
print (res)

Working Domain Specified Successfully, with domainId:0201adc4-ae96-46f0-ae3d-01cdba9e41d6


In [224]:
Get_all_devices_in_domain_url = nb_url + "/ServicesAPI/API/V1/CMDB/Devices"

def get_all_devices(Get_all_devices_in_domain_url, headers, token):
    try:
        response = requests.get(Get_all_devices_in_domain_url, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            devices = result["devices"]
            return devices
        else:
            return("Get Devices failed! - " + str(response.text))
    except Exception as e:
        return (str(e)) 
    
devices = get_all_devices(Get_all_devices_in_domain_url, headers, token)
devices

[{'id': 'cd97d9ce-1d39-421d-a56d-e8da3aaa08c7',
  'mgmtIP': '10.1.13.2',
  'hostname': 'Client2',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': '77242378-e865-449e-adeb-c4eeaf361853',
  'mgmtIP': '10.1.14.2',
  'hostname': 'Client3',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': '575fd214-acdf-427c-914a-2acd2aedb6af',
  'mgmtIP': '123.12.12.12',
  'hostname': 'R12',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': '4b814621-05d0-4dd2-98e0-59d79b1ec410',
  'mgmtIP': '123.11.11.11',
  'hostname': 'R11',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': '432d39b7-9729-4313-af18-3cbbf8473663',
  'mgmtIP': '123.20.1.3',
  'hostname': 'SW5',
  '

In [225]:
ips = []

for i in range(len(devices)):
    ip = devices[i]["mgmtIP"]
    ips.append(ip)
    
ips

['10.1.13.2',
 '10.1.14.2',
 '123.12.12.12',
 '123.11.11.11',
 '123.20.1.3',
 '10.18.19.19',
 '10.1.14.1',
 '123.204.4.4',
 '123.203.3.3',
 '10.1.13.1',
 '10.120.13.1',
 '123.10.1.2',
 '123.10.1.6',
 '10.120.15.5',
 '123.20.1.1',
 '123.8.8.8',
 '123.10.1.22',
 '123.20.1.18',
 '123.7.7.7',
 '123.20.1.11',
 '123.10.1.18',
 '123.9.9.9',
 '123.10.10.10',
 '123.20.1.2',
 '10.18.19.18',
 '10.120.100.2']

In [226]:
Resolve_Device_Gateway_url = nb_url + "/ServicesAPI/API/V1/CMDB/Path/Gateways"
results = []
def resolve_device_gateway(Resolve_Device_Gateway_url, token, ipOrHost, headers):
    headers["Token"] = token
    data = {"ipOrHost":ipOrHost}
    try:
        response = requests.get(Resolve_Device_Gateway_url, params = data, headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Create module attribute failed! - " + str(response.text))

    except Exception as e:
        print (str(e)) 
        
for i in range(len(ips)):
    result = resolve_device_gateway(Resolve_Device_Gateway_url, token, ips[i], headers)
    results.append(result["gatewayList"])

#result = resolve_device_gateway(Resolve_Device_Gateway_url, token, ips[0], headers)
#result["gatewayList"]
results

[[{'ip': '10.1.13.2', 'devName': 'Client2', 'intfName': 'Ethernet0/0'}],
 [{'ip': '10.1.14.2', 'devName': 'Client3', 'intfName': 'Ethernet0/0'}],
 [{'ip': '123.12.12.12', 'devName': 'R12', 'intfName': 'Loopback0'}],
 [{'ip': '123.11.11.11', 'devName': 'R11', 'intfName': 'Loopback0'}],
 [{'ip': '123.20.1.3', 'devName': 'SW5', 'intfName': 'Vlan55'}],
 [{'ip': '10.18.19.19', 'devName': 'R19', 'intfName': 'Tunnel0'}],
 [{'ip': '10.1.14.1', 'devName': 'R14', 'intfName': 'Ethernet0/0'}],
 [{'ip': '123.204.4.4', 'devName': 'SW4', 'intfName': 'Loopback0'}],
 [{'ip': '123.203.3.3', 'devName': 'SW3', 'intfName': 'Loopback0'}],
 [{'ip': '10.1.13.1', 'devName': 'R13', 'intfName': 'Ethernet0/0'}],
 [{'ip': '10.120.13.1', 'devName': 'R2', 'intfName': 'Ethernet0/3.13'}],
 [{'ip': '123.10.1.2', 'devName': 'R1', 'intfName': 'Ethernet0/2'}],
 [{'ip': '123.10.1.6', 'devName': 'R5', 'intfName': 'Ethernet0/1'}],
 [{'ip': '10.120.15.5', 'devName': 'R3', 'intfName': 'Ethernet0/3.15'}],
 [{'ip': '123.20.1.1',

In [227]:


Calculate_Path_url = nb_url + "/ServicesAPI/API/V1/CMDB/Path/Calculation"

def calculate_path(Calculate_Path_url, body, headers, token):
    headers["Token"] = token
    
    try:
        response = requests.post(Calculate_Path_url, data = json.dumps(body), headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result["taskID"])
        else:
            return ("Create module attribute failed! - " + str(response.text))

    except Exception as e:
        return (str(e)) 

ress = []    
for i in range(len(results) - 1):
    gw = results[i][0]
    #gw1 = results[i+1][0]

    sourceIP = gw["ip"]
    sourcePort = 0
    sourceGwIP = gw["ip"]
    sourceGwDev = gw["devName"]
    sourceGwIntf = gw["intfName"]
    destIP = "10.120.100.2"
    destPort = 0
    pathAnalysisSet = 2
    protocol = 4
    isLive = False

    body = {
                "sourceIP" : sourceIP,                # IP address of the source device.
                "sourcePort" : sourcePort,
                "sourceGwDev" : sourceGwDev,          # Hostname of the gateway device.
                "sourceGwIP" : sourceGwIP,            # Ip address of the gateway device.
                "sourceGwIntf" : sourceGwIntf,        # Name of the gateway interface.
                "destIP" : destIP,                    # IP address of the destination device.
                "destPort" : destPort,
                "pathAnalysisSet" : pathAnalysisSet,  # 1:L3 Path; 2:L2 Path; 3:L3 Active Path
                "protocol" : protocol,                # Specify the application protocol, check online help, such as 4 for IPv4.
                "isLive" : isLive                     # False: Current Baseline; True: Live access
        } 

    res = calculate_path(Calculate_Path_url, body, headers, token)
    ress.append(res)
ress


['238d9ecc-8cdd-499c-8c52-e43295947cdb',
 '69b909a1-c44d-4ff7-ace7-5defd839a577',
 '18b86e07-5a71-42ec-8895-5d6611b6d0bf',
 '50e1d0b6-0a12-458b-9e5d-9a39716ffe19',
 '32761ae0-16e0-4508-8f06-3b578ed68267',
 '5ba252be-34c8-4b0f-a72b-483f3b1d82b0',
 '5a1d9d59-0e1e-4fa4-acad-6ce36f952e39',
 'b30f51a2-03ef-45ee-ac0a-c3675fd7ddca',
 'b4bdb963-6570-4236-a4e5-91af34b3da28',
 '9292cca9-9289-441b-a077-a2a1ed7d6b6a',
 '1bca8a05-cbe4-4261-afd4-b00452aff606',
 'd5d851b7-1206-476a-8383-afd6383006ce',
 '2a383704-99b3-49f9-b76b-3e252320e188',
 '458a6b2e-7446-4093-ab01-c7ecf80036bd',
 '870a4153-ec3a-4547-ad63-e5c1c8e8a4ea',
 '341a3c5d-6bae-47a4-b607-e440e9453ab3',
 '9364d0d3-310a-461b-9e7c-e6c2734538ac',
 '22d4dd87-a765-47fe-a0b0-132684c31f85',
 'a57391da-53e1-43ec-b814-bf980fd89fd3',
 'e0801d92-b4cc-42c9-9e0a-779562d002c5',
 '934c04d9-eb34-48b8-93c5-6892ac801b6f',
 'a6b95af7-aea3-4611-988d-28a135de347a',
 '7f42f5ae-9b44-4805-826b-460996cefa02',
 '301d3747-a5d9-40a2-b39d-24d73c1db271',
 '39118681-34ac-

In [232]:
sourceIP = "10.120.15.5"
sourcePort = 0
sourceGwIP = "10.120.15.5"
sourceGwDev = "R3"
sourceGwIntf = "Ethernet0/3.15"
destIP = "10.120.13.1"
destPort = 0
pathAnalysisSet = 1
protocol = 4
isLive = False

body = {
            "sourceIP" : sourceIP,                # IP address of the source device.
            "sourcePort" : sourcePort,
            "sourceGwDev" : sourceGwDev,          # Hostname of the gateway device.
            "sourceGwIP" : sourceGwIP,            # Ip address of the gateway device.
            "sourceGwIntf" : sourceGwIntf,        # Name of the gateway interface.
            "destIP" : destIP,                    # IP address of the destination device.
            "destPort" : destPort,
            "pathAnalysisSet" : pathAnalysisSet,  # 1:L3 Path; 2:L2 Path; 3:L3 Active Path
            "protocol" : protocol,                # Specify the application protocol, check online help, such as 4 for IPv4.
            "isLive" : isLive                     # False: Current Baseline; True: Live access
    } 

resUnique = calculate_path(Calculate_Path_url, body, headers, token)
resUnique

'da5c1a2f-cee7-4e3b-a7e9-1cac5c6458a1'

In [229]:
import time
#time.sleep(30)   # Delays for 5 seconds. You can also use a float value.
def get_patth_result(Get_Path_Calulation_Result_url, headers, token):
    headers["Token"] = token
    try:
        response = requests.get(Get_Path_Calulation_Result_url, headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result.json())
        else:
            return (response.json())

    except Exception as e:
        return (str(e)) 

Final = []
for i in range(len(ress)):
    time.sleep(3)   # Delays for 5 seconds. You can also use a float value.
    Get_Path_Calulation_Result_url = nb_url + "/ServicesAPI/API/V1/CMDB/Path/Calculation/" + str(ress[i]) + "/Result"
    final = get_patth_result(Get_Path_Calulation_Result_url, headers, token)
    d = json.dumps(final)
    j = json.loads(d)
    '''print("##################################################################################################################")
    print(str(i) + " " + ress[i])
    print("--------------------------------------------------------------------------------------------------------------------")
    statusD = j["statusDescription"]
    return(statusD)
    print("--------------------------------------------------------------------------------------------------------------------")'''
    Final.append(final)
'''for j in range(len(Final)):
    f = Final[j]
    print(str(j) + " " + ress[j]) 
    print (f)
    '''

Final

[{'hopList': [{'hopId': '278a5540-6dd4-4f19-97d6-2524decff1c0',
    'srcDeviceName': 'Client2',
    'inboundInterface': 'Ethernet0/0',
    'mediaName': '',
    'dstDeviceName': 'R13',
    'outboundInterface': 'Ethernet0/0',
    'nextHopIdList': []}],
  'statusCode': 794008,
  'statusDescription': "Task '238d9ecc-8cdd-499c-8c52-e43295947cdb' is failed"},
 {'hopList': [{'hopId': '2811b11b-b610-4040-9b4b-a84d1cba5d54',
    'srcDeviceName': 'Client3',
    'inboundInterface': 'Ethernet0/0',
    'mediaName': '',
    'dstDeviceName': 'R14',
    'outboundInterface': 'Ethernet0/0',
    'nextHopIdList': ['87e2098f-ff1d-49f7-953a-ec56ae24d218']},
   {'hopId': '87e2098f-ff1d-49f7-953a-ec56ae24d218',
    'srcDeviceName': 'R14',
    'inboundInterface': 'Ethernet0/1',
    'mediaName': '',
    'dstDeviceName': '202.2.14.1',
    'outboundInterface': 'Ethernet0',
    'nextHopIdList': []}],
  'statusCode': 794008,
  'statusDescription': "Task '69b909a1-c44d-4ff7-ace7-5defd839a577' is failed"},
 {'hopList

In [233]:
Get_Path_Calulation_Result_url = nb_url + "/ServicesAPI/API/V1/CMDB/Path/Calculation/" + str(resUnique) + "/Result"
def get_patth_result(Get_Path_Calulation_Result_url, headers, token):
    headers["Token"] = token
    try:
        response = requests.get(Get_Path_Calulation_Result_url, headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return (response.text)

    except Exception as e:
        return (str(e)) 

res = get_patth_result(Get_Path_Calulation_Result_url, headers, token)
res

{'hopList': [{'hopId': '748ee44a-7fcd-4b5c-9b5d-9f63389ab2aa',
   'srcDeviceName': 'R3',
   'inboundInterface': 'Ethernet0/3.15',
   'mediaName': '10.120.15.4/30',
   'dstDeviceName': 'R20',
   'outboundInterface': 'Ethernet3/0.15',
   'nextHopIdList': ['24134b6c-9074-4763-84c7-b785f2819187']},
  {'hopId': '24134b6c-9074-4763-84c7-b785f2819187',
   'srcDeviceName': 'R20',
   'inboundInterface': 'Ethernet2/0.13',
   'mediaName': '10.120.13.0/30',
   'dstDeviceName': 'R2',
   'outboundInterface': 'Ethernet0/3.13',
   'nextHopIdList': []}],
 'statusCode': 790200,
 'statusDescription': 'Success.'}

In [231]:
import os
import sys

def pause():
    
    programPause = raw_input("Press the <ENTER> key to continue...")

print("Think about what you ate for dinner last night...")
pause()

Think about what you ate for dinner last night...


NameError: name 'raw_input' is not defined

In [ ]:
Logout_url = nb_url + "/ServicesAPI/API/V1/Session"

def logout(Logout_url, token, headers):
    headers["token"] = token
    
    try:
        # Do the HTTP request
        response = requests.delete(Logout_url, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js)
        else:
            return ("Session logout failed! - " + str(response.text))

    except Exception as e:
        return (str(e))

logout = logout(Logout_url, token, headers)
logout